In [70]:
from rdflib import Graph, URIRef, Namespace, BNode, RDFS
import httpx

In [2]:
uri = "http://id.loc.gov/resources/instances/1893399"
graph = Graph()
graph.parse(f'{uri}.rdf')
# graph.serialize("instance.ttl")

<Graph identifier=N1ec807ea5d144858b45b0f69a83ec51b (<class 'rdflib.graph.Graph'>)>

In [73]:
from pydantic import BaseModel
from typing import Optional
from api.src.schemas.metadata.bibframe.adminMetadata import AdminMetadata
from api.src.schemas.metadata.bibframe.title import Title
from api.src.schemas.metadata.bibframe.element import Element

class Valeu(BaseModel):
    label: str
    type: str

class ProvisionActivity(BaseModel):
    agent: Optional[str]
    date: str
    place: str

class Instance(BaseModel):
    adminMetadata: AdminMetadata
    type: list[str]
    title: Title
    carrier: list[Element]
    copyrightDate: Optional[str]
    dimensions: Optional[str]
    extent: Optional[Valeu]
    instanceOf: list[Element]
    issuance: Optional[list[Element]]
    media: Optional[list[Element]]
    provisionActivity: list[ProvisionActivity]
    provisionActivityStatement: Optional[str]
    responsibilityStatement: Optional[str]
    seriesStatement: Optional[str]


In [69]:
from api.src.function.bibframe.bf_provisionActivity import GetProvisionActivity
from api.src.function.bibframe.bf_type import GetType
from api.src.function.bibframe.bf_title import GetTitle
from api.src.function.bibframe.bf_Uri import GetUriBF


def GetElement(graph, bf, obj):
    q = f"""PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            SELECT ?type ?label
                WHERE {{ 
                    <{uri}> bf:{bf} ?uri .
                    ?uri rdfs:label ?label .
                    ?uri rdf:type ?type .
                    }}"""
    response = graph.query(q)
    bindings = response.bindings
    if len(bindings) > 0:
        binding = bindings[0]
        v = {
            "type": binding.get('type').toPython(),
            "label": binding.get('label').toPython()
        }
        obj[bf] = v
    return obj

def GetValeu(graph, uri, bf, obj):
    q = f"""PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            SELECT ?value
                WHERE {{ 
                    <{uri}> bf:{bf} ?value .
                    }}"""
    response = graph.query(q)
    bindings = response.bindings
    if len(bindings) > 0:
        binding = bindings[0]
        obj[bf] = binding.get('value').toPython()
    return obj

def ParserInstance(graph, uri):
    identifier = uri.split("/")[-1]
    types = GetType(graph, uri)
    title = GetTitle(graph, uri)

    obj = {
    'adminMetadata': {
        'generationProcess': {
            'label': 'BiblioKeia'
        },
        'identifiedBy': [ 
            {
                "type": "Local",
            "assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
            "value": identifier
            }
        ]
    },
    "type": types,
    'title': title 
    }
    # carrier
    obj = GetUriBF(graph, uri, 'carrier', obj)
    # copyrightDate
    # obj = GetValeu(graph, "copyrightDate", obj)
    # dimensions
    obj = GetValeu(graph, "dimensions", obj)
    # extent
    obj = GetElement(graph, "extent", obj)
    # instanceOf
    obj = GetUriBF(graph, uri, 'instanceOf', obj)
    # issuance
    obj = GetUriBF(graph, uri, 'issuance', obj)
    # media
    obj = GetUriBF(graph, uri, 'media', obj)
    # ProvisionActivity
    obj = GetProvisionActivity(graph, uri, obj)
    # provisionActivityStatement
    obj = GetValeu(graph, "provisionActivityStatement", obj)
    # responsibilityStatement
    obj = GetValeu(graph, "responsibilityStatement", obj)
    # seriesStatement
    obj = GetValeu(graph, "seriesStatement", obj)

    response = Instance(**obj)
    
    return response

response = ParserInstance(graph, uri)
response.provisionActivity

[ProvisionActivity(agent=None, date='"1968"^^<http://id.loc.gov/datatypes/edtf>', place='New York (State)'),
 ProvisionActivity(agent='Harper & Row', date='"1968, c1965"', place='New York')]

In [11]:
u = URIRef('http://id.loc.gov/resources/instances/1893399')
bf = Namespace("http://id.loc.gov/ontologies/bibframe/")

In [115]:
r = graph.triples((u, bf.provisionActivity, None))
for _, _, o in r:
    provisionActivity = graph.triples((o, None, None))
    objPA = {}
    for s, p, o in provisionActivity:
            if p == bf.agent:
                if isinstance(o, BNode):
                    agent = graph.triples((o, RDFS.label, None))
                    for _, _, label in agent:
                        objPA['agent'] = label.value
            elif p == bf.date:
                objPA['date'] = o.n3()
            elif p == bf.place:
                place = graph.triples((o, RDFS.label, None))
                for _, _, label in place:
                    objPA['place'] = label.value
    print(objPA)
    break

{'date': '"1968"^^<http://id.loc.gov/datatypes/edtf>', 'place': 'New York (State)'}


In [71]:
response = httpx.get("http://localhost:8000/import/loc/instances?uri=http%3A%2F%2Fid.loc.gov%2Fresources%2Finstances%2F1893399")
response

<Response [200 OK]>

In [ ]:
response.json()

In [74]:
request = Instance(**response.json())
request

Instance(adminMetadata=AdminMetadata(encodingLevel='menclvl:f', assigner='http://id.loc.gov/vocabulary/organizations/brmninpa', creationDate='2023-06-16', descriptionConventions='http://id.loc.gov/vocabulary/descriptionConventions/isbd', descriptionModifier='http://id.loc.gov/vocabulary/organizations/brmninpa', descriptionLanguage='http://id.loc.gov/vocabulary/languages/por', generationProcess=GenerationProcess(label='BiblioKeia', generationDate='2023-06-16T14:05:52'), identifiedBy=[IdentifiedBy(type='Local', assigner='http://id.loc.gov/vocabulary/organizations/dlc', value='1893399')], status=Status(value='mstatus:new', label='novo')), type=['Instance', 'Print'], title=Title(type='bf:Title', mainTitle='Conjectures and refutations', subtitle='the growth of scientific knowledge', label='Conjectures and refutations : the growth of scientific knowledge'), carrier=[Element(label='volume', lang=None, uri='http://id.loc.gov/vocabulary/carriers/nc', type=['http://id.loc.gov/ontologies/bibframe

In [84]:
request.extent

Valeu(label='xiii, 417 p.', type='http://id.loc.gov/ontologies/bibframe/Extent')

In [85]:
def MakeLiteral(value):
    v = f"""bf:extent [ a {value.type} ;
                    rdfs:label "{value.label}" ] ;"""
    return v
    
  
MakeLiteral(request.extent) 

'bf:extent [ a http://id.loc.gov/ontologies/bibframe/Extent ;\n                    rdfs:label "xiii, 417 p." ] ;'

In [ ]:
from api.src.function.bibframe.Work.graphWork import MakeUri


prefix = """
PREFIX bki: <https://bibliokeia.com/resources/instance/> 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    PREFIX bflc: <http://id.loc.gov/ontologies/bflc/>  
    PREFIX menclvl: <http://id.loc.gov/vocabulary/menclvl/>
    PREFIX mstatus: <http://id.loc.gov/vocabulary/mstatus/>
    PREFIX contentTypes: <http://id.loc.gov/vocabulary/contentTypes/>
    PREFIX relators: <http://id.loc.gov/vocabulary/relators/>
    PREFIX genreForms: <http://id.loc.gov/authorities/genreForms/>
    PREFIX msupplcont: <http://id.loc.gov/vocabulary/msupplcont/>
    PREFIX millus: <http://id.loc.gov/vocabulary/millus/>    
    """

def MakeGraphInstance(request, id):
    graph = f"""{prefix}    
    INSERT DATA {{
        GRAPH bki:{id}
        {{
                bki:{id} a { ", ".join([f'bf:{i}' for i in request.type]) }  ;
                bf:adminMetadata [ a bf:AdminMetadata ;
                bflc:encodingLevel {request.adminMetadata.encodingLevel} ;
                bf:assigner <{request.adminMetadata.assigner}> ;    
                bf:creationDate "{request.adminMetadata.creationDate}"^^xsd:date ;    
                bf:descriptionConventions <{request.adminMetadata.descriptionConventions}> ;
                bf:descriptionLanguage <{request.adminMetadata.descriptionLanguage}> ;
                 bf:generationProcess [ a bf:GenerationProcess ;
                    rdfs:label "{request.adminMetadata.generationProcess.label}" ;
                    bf:generationDate "{request.adminMetadata.generationProcess.generationDate}"^^xsd:dateTime ] ;
                bf:identifiedBy [ a bf:Local ;
                    bf:assigner <{request.adminMetadata.assigner}> ;
                    rdf:value "{id}" ] ;
                bf:status {request.adminMetadata.status.value} ] ;            
                bf:title [ a bf:Title ;
                bf:mainTitle "{request.title.mainTitle}" 
                { f'; bf:subtitle "{request.title.subtitle}" ' if request.title.subtitle else ''} ] ;
                { MakeUri("carrier", request.carrier) if request.carrier else "" }
                { MakeUri("copyrightDate", request.copyrightDate) if request.copyrightDate else "" }
                { f'bf:dimensions "{request.dimensions}" ; ' if request.dimensions else ""  }
                { MakeLiteral(request.extent) if request.extent else "" }
                { MakeUri("instanceOf", request.instanceOf) if request.instanceOf else "" }
        }} }}
        """
    return graph

G = MakeGraphInstance(request, "TESTE")
print(G)

In [87]:
from api.src.schemas.settings import Settings
from pyfuseki import FusekiUpdate

settings = Settings()

collectionUpdate = FusekiUpdate(f'{settings.url}:3030', 'collection')

In [97]:
request.dict()

{'adminMetadata': {'encodingLevel': 'menclvl:f',
  'assigner': 'http://id.loc.gov/vocabulary/organizations/brmninpa',
  'creationDate': '2023-06-16',
  'descriptionConventions': 'http://id.loc.gov/vocabulary/descriptionConventions/isbd',
  'descriptionModifier': 'http://id.loc.gov/vocabulary/organizations/brmninpa',
  'descriptionLanguage': 'http://id.loc.gov/vocabulary/languages/por',
  'generationProcess': {'label': 'BiblioKeia',
   'generationDate': '2023-06-16T14:05:52'},
  'identifiedBy': [{'type': 'Local',
    'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
    'value': '1893399'}],
  'status': {'value': 'mstatus:new', 'label': 'novo'}},
 'type': ['Instance', 'Print'],
 'title': {'type': 'bf:Title',
  'mainTitle': 'Conjectures and refutations',
  'subtitle': 'the growth of scientific knowledge',
  'label': 'Conjectures and refutations : the growth of scientific knowledge'},
 'carrier': [{'label': 'volume',
   'lang': None,
   'uri': 'http://id.loc.gov/vocabulary/car

In [107]:
request.provisionActivity

[ProvisionActivity(agent=None, date='"1968"^^<http://id.loc.gov/datatypes/edtf>', place='New York (State)'),
 ProvisionActivity(agent='Harper & Row', date='"1968, c1965"', place='New York')]

In [106]:
def DocInstance(request, id):

    doc = {
        "id": id,
        "type": request.type,
        "mainTitle": request.title.mainTitle,
        "subtitle": request.title.subtitle,
        "carrier": [i.label for i in request.carrier],
        "dimensions": request.dimensions,
        "extent": request.extent.label if request.extent else None,
        "issuance": [i.label for i in request.issuance],
        "media": [i.label for i in request.media],


        
        }
    

    # responseSolr =  solr.add([doc], commit=True)

    return doc 

DocInstance(request, "TESTE")


{'id': 'TESTE',
 'type': ['Instance', 'Print'],
 'mainTitle': 'Conjectures and refutations',
 'subtitle': 'the growth of scientific knowledge',
 'carrier': ['volume'],
 'dimensions': '21 cm.',
 'extent': 'xiii, 417 p.',
 'issuance': ['single unit'],
 'media': ['unmediated']}